In [10]:
from IPython.core.display import display, HTML
from warnings import filterwarnings
display(HTML("<style>.container { width:100% !important; }</style>"))
filterwarnings(action='ignore')

In [54]:
# evalys requires pandas==0.25.3
# %matplotlib inli|ne
import pandas as pd
import matplotlib
from evalys.jobset import JobSet

In [55]:
input_location = 'out'

In [56]:
schedule = pd.read_csv('../output/' + input_location + '_schedule.csv')
schedule.T

,0
batsim_version,4.0.0
consumed_joules,0
makespan,866913
max_slowdown,1.00003
max_turnaround_time,16150.3
max_waiting_time,0
mean_slowdown,0.98865
mean_turnaround_time,46.694
mean_waiting_time,0
nb_computing_machines,109


In [24]:
js = JobSet.from_csv('../output/' + input_location + '_jobs.csv')
js.df.drop(['consumed_energy'], axis=1, inplace=True)
df = js.df.sort_values(by=['submission_time'])
df

,jobID,workload_name,profile,submission_time,requested_number_of_resources,requested_time,success,final_state,starting_time,execution_time,finish_time,waiting_time,turnaround_time,stretch,allocated_resources,metadata,proc_alloc
0,15,w0,15,1.000000,4,53940.000000,0,REJECTED,NaN,NaN,NaN,NaN,NaN,NaN,[],961904786.0,0
1,0,w0%15,data_staging_profile_0,1.000000,2,53940.000000,1,COMPLETED_SUCCESSFULLY,1.000000,0.769624,1.769624,0.0,0.769624,1.0,"[96, 108]",NaN,2
2,1,w0%15,data_staging_profile_0,1.000000,2,53940.000000,1,COMPLETED_SUCCESSFULLY,1.000000,0.769624,1.769624,0.0,0.769624,1.0,"[96, 108]",NaN,2
3,2,w0%15,data_staging_profile_0,1.000000,2,53940.000000,1,COMPLETED_SUCCESSFULLY,1.000000,0.769624,1.769624,0.0,0.769624,1.0,"[96, 108]",NaN,2
4,3,w0%15,data_staging_profile_0,1.000000,2,53940.000000,1,COMPLETED_SUCCESSFULLY,1.000000,0.769624,1.769624,0.0,0.769624,1.0,"[96, 108]",NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88047,1552,w0%1006,data_staging_profile_0,866888.216928,2,4261.090219,1,COMPLETED_SUCCESSFULLY,866888.216928,24.759736,866912.976664,0.0,24.759736,1.0,"[98, 108]",NaN,2
88046,1551,w0%1006,data_staging_profile_0,866888.216928,2,4261.090219,1,COMPLETED_SUCCESSFULLY,866888.216928,24.759736,866912.976664,0.0,24.759736,1.0,"[98, 108]",NaN,2
88045,1550,w0%1006,data_staging_profile_0,866888.216928,2,4261.090219,1,COMPLETED_SUCCESSFULLY,866888.216928,24.759736,866912.976664,0.0,24.759736,1.0,"[96, 108]",NaN,2
88063,1568,w0%1006,data_staging_profile_0,866888.216928,2,4261.090219,1,COMPLETED_SUCCESSFULLY,866888.216928,24.759736,866912.976664,0.0,24.759736,1.0,"[100, 108]",NaN,2


In [25]:
timeout_jobs = df[df.final_state == 'COMPLETED_WALLTIME_REACHED']['workload_name'].unique()
timeout_jobs

array(['w0%62', 'w0%72', 'w0%70', 'w0%84', 'w0%125', 'w0%127', 'w0%128',
       'w0%129', 'w0%144', 'w0%200', 'w0%197', 'w0%262', 'w0%260',
       'w0%301', 'w0%300', 'w0%376', 'w0%406', 'w0%447', 'w0%433',
       'w0%511', 'w0%448', 'w0%782', 'w0%813'], dtype=object)

In [26]:
len(timeout_jobs)

23

In [27]:
dyn = df[df.final_state != 'REJECTED']

In [28]:
len(dyn.workload_name.unique())

979

In [29]:
static = df[df.final_state == 'REJECTED']

In [30]:
dyn_times = dyn[['workload_name', 'starting_time', 'execution_time']].groupby('workload_name').agg({'starting_time': 'min', 'execution_time': 'sum'})

In [31]:
dyn_times.execution_time.mean()

4202.223275303367

In [63]:
comp_times = dyn[dyn.profile == 'parallel_homogeneous_profile_0'][['workload_name', 'execution_time']].groupby('workload_name').sum()
comp_times

,execution_time
workload_name,
w0%100,168.75000
w0%1000,12.00020
w0%1001,9.00020
w0%1002,42.75020
w0%1003,6770.25350
...,...
w0%995,1.50020
w0%996,6086.25000
w0%997,2.25015


In [68]:
start_end_times = dyn[['workload_name', 'starting_time', 'finish_time']].groupby('workload_name').agg({'starting_time': 'min', 'finish_time':'max'})
total_times = start_end_times.finish_time - start_end_times.starting_time
total_times = total_times.to_frame('execution_time')
total_times

,execution_time
workload_name,
w0%100,170.415504
w0%1000,19.667592
w0%1001,9.240900
w0%1002,83.993814
w0%1003,6844.232050
...,...
w0%995,30.692516
w0%996,6100.901919
w0%997,8.293718


In [78]:
all_times = total_times.join(comp_times, lsuffix='_total', rsuffix='_comp')
all_times['execution_time_io'] = all_times.execution_time_total - all_times.execution_time_comp
all_times['ratio'] = all_times.execution_time_comp / all_times.execution_time_io
all_times

,execution_time_total,execution_time_comp,execution_time_io,ratio
workload_name,,,,
w0%100,170.415504,168.75000,1.665504,101.320681
w0%1000,19.667592,12.00020,7.667392,1.565095
w0%1001,9.240900,9.00020,0.240700,37.391774
w0%1002,83.993814,42.75020,41.243614,1.036529
w0%1003,6844.232050,6770.25350,73.978550,91.516440
...,...,...,...,...
w0%995,30.692516,1.50020,29.192316,0.051390
w0%996,6100.901919,6086.25000,14.651919,415.389274
w0%997,8.293718,2.25015,6.043568,0.372321


In [79]:
all_times.describe()

,execution_time_total,execution_time_comp,execution_time_io,ratio
count,979.000000,977.000000,977.000000,977.000000
mean,3293.551756,3265.752989,34.418115,179.993194
std,13082.958305,13081.965704,63.749579,1218.286375
min,2.485176,1.500000,0.049861,0.047421
25%,22.783229,12.750100,3.619537,1.247840
50%,59.033779,27.750300,12.695169,6.933809
75%,664.526222,649.500900,36.067758,45.519504
max,161584.196187,161502.752500,670.246477,34152.971809
